In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from time import time
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Embedding, LSTM, Dense, Dot, Reshape, Lambda
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import Layer
from keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
from gensim.models import KeyedVectors
import numpy as np
from keras.preprocessing.text import Tokenizer
import itertools
import re

def split_and_zero_padding(df, max_seq_length):
    # Split to dicts
    X = {'left': df['left_n'], 'right': df['right_n']}
    # Zero padding
    for dataset, side in itertools.product([X], ['left', 'right']):
        dataset[side] = pad_sequences(dataset[side], padding='pre', truncating='post', maxlen=max_seq_length)
    return dataset

#  --

def clean_data(file):
    lines = [0]*2
    lines[0] = []
    lines[1] = []
    with open(file, "r", encoding="utf-8") as f:
        for line in f:
            line = line.lower()
            line = line.rstrip().split("\t")
            doc_w = []
            doc_w.append(line[0].split())
            doc_w.append(line[1].split())
            #line2 = []
            for i in range(2):
                words = []
                for w in doc_w[i]:
                    w = w.replace("'s", '')
                    w = re.sub(re.compile("[!-/:-@[-`{-~]"), '', w)
            #    line2.append(w)
            #stemmer = PorterStemmer()
            #line = list(map(stemmer.stem, line2))
            #wnl = WordNetLemmatizer()
            #line = list(map(wnl.lemmatize, line2))
            #for w in line:
                    if w not in stop_words:
                        if w != '':
                            words.append(w)
                lines[i].append(words)
    return lines

def clean_data2(file):
    words = []
    with open(file, "r", encoding="utf-8") as f:
        for line in f:
            line = line.lower()
            line = line.rstrip().split()
            #line2 = []
            for w in line:
                w = w.replace("'s", '')
                w = re.sub(re.compile("[!-/:-@[-`{-~]"), '', w)
            #    line2.append(w)
            #stemmer = PorterStemmer()
            #line = map(stemmer.stem, line2)
            #wnl = WordNetLemmatizer()
            #line = map(wnl.lemmatize, line)
            #for w in line:
                if w not in stop_words:
                    if w != '':
                        words.append(w)
    return words

dataset = [
'./drive/MyDrive/Colab Notebooks/training_data/STS.input.OnWN.txt',
'./drive/MyDrive/Colab Notebooks/add_data/STS.input.deft-forum.txt',
'./drive/MyDrive/Colab Notebooks/add_data/STS.input.deft-news.txt',
'./drive/MyDrive/Colab Notebooks/add_data/STS.input.headlines.txt',
'./drive/MyDrive/Colab Notebooks/add_data/STS.input.MSRpar.txt',
'./drive/MyDrive/Colab Notebooks/add_data/STS.input.MSRvid.txt',
'./drive/MyDrive/Colab Notebooks/add_data/STS.input.SMTeuroparl.txt',
'./drive/MyDrive/Colab Notebooks/add_data/STS.input.tweet-news.txt',
]
q0 = []
q1 = []
file = []
for fname in dataset:
    questions = clean_data(fname)
    q0.extend(questions[0])
    q1.extend(questions[1])
    file.append(clean_data2(fname))
train_df = pd.DataFrame({'left' : [], 'right' : []})
train_df['left'] = np.array(q0)
train_df['right'] = np.array(q1)

# Load training set
#for q in ['left', 'right']:
#    train_df[q + '_n'] =
 

tokenizer_L = Tokenizer()
tokenizer_R = Tokenizer()
tokenizer_L.fit_on_texts(train_df['left'])
tokenizer_R.fit_on_texts(train_df['right'])
#dic_L = tokenizer_L.word_index
#dic_R = tokenizer_R.word_index
#words_L = []
#words_R = []
#for list in train_df['left']:
#    words = []
#    for w in list:
#        words.append(dic_L[w])
#    words_L.append(words)
#for list in train_df['right']:
#    words = []
#    for w in list:
#        words.append(dic_R[w])
#    words_R.append(words)
#train_df['left_n'] = words_L
#train_df['right_n'] = words_R
train_df['left_n'] = tokenizer_L.texts_to_matrix(train_df['left'], "binary")
train_df['right_n'] = tokenizer_R.texts_to_matrix(train_df['right'], "binary")
print(train_df['left_n'])
print(train_df['right_n'])

#count1 = 0
#for list in train_df['left']:
#    count2 = 0
#    for word in list:
#        embeddings[train_df['left'][count1][count2]] = matrix[word]
#        count2 += 1
#    count1 += 1
#print(embeddings)

embedding_dim = 200
max_seq_length = 20
#use_w2v = True
#train_df, embeddings = make_w2v_embeddings(train_df, file, embedding_dim=embedding_dim, empty_w2v=not use_w2v)

# Split to train validation
validation_size = int(len(train_df) * 0.1)
training_size = len(train_df) - validation_size

X = train_df[['left_n', 'right_n']]

#left = []
#right = []
#for list in train_df['left'].values:
#    left.append(np.array(list))
#for list in train_df['right'].values:
#    right.append(np.array(list))
#X_train = {'left': np.array(left), 'right': np.array(right)}
#print(X_train)

def gs_label(file):
    labels = []
    with open(file, "r", encoding="utf-8") as f:
        for line in f:
            line = line.rstrip().split()
            labels.append(line)
    return labels

train_gs = [
'./drive/MyDrive/Colab Notebooks/training_data/STS.gs.OnWN.txt',
'./drive/MyDrive/Colab Notebooks/add_data/STS.gs.deft-forum.txt',
'./drive/MyDrive/Colab Notebooks/add_data/STS.gs.deft-news.txt',
'./drive/MyDrive/Colab Notebooks/add_data/STS.gs.headlines.txt',
'./drive/MyDrive/Colab Notebooks/add_data/STS.gs.MSRpar.txt',
'./drive/MyDrive/Colab Notebooks/add_data/STS.gs.MSRvid.txt',
'./drive/MyDrive/Colab Notebooks/add_data/STS.gs.SMTeuroparl.txt',
'./drive/MyDrive/Colab Notebooks/add_data/STS.gs.tweet-news.txt'
]
train_labels = []
for fname in train_gs:
    train_labels.extend(gs_label(fname))

X_train, X_validation, Y_train, Y_validation = train_test_split(X, train_labels, test_size=validation_size)
X_train = split_and_zero_padding(X_train, max_seq_length)
X_validation = split_and_zero_padding(X_validation, max_seq_length)

y_t = []
for list in Y_train:
    for l in list:
        y_t.append(float(l)/5)
Y_train = np.array(y_t)

y_v = []
for list in Y_validation:
    for l in list:
        y_v.append(float(l)/5)
Y_validation = np.array(y_v)

#y_t = []
#for gs in train_labels:
#    gs = str(gs)
#    gs = gs.replace("['", '')
#    gs = gs.replace("']", '')
#    y_t.append(float(gs)/5)
#Y_train = np.array(y_t)

# --
# Model variables
batch_size = 64
n_epoch = 50
n_hidden = 600

left_input = Input(shape=(max_seq_length,))
right_input = Input(shape=(max_seq_length,))

embedding_layer = Embedding(len(train_df), embedding_dim, mask_zero=True, 
                            input_shape=(max_seq_length,), trainable=True)
x_left = embedding_layer(left_input)
x_right = embedding_layer(right_input)

#LSTM
shared_lstm = LSTM(n_hidden, dropout=0.5, return_state=True)
a, b, encoded_left = shared_lstm(x_left)
a, b, encoded_right = shared_lstm(x_right)

cos_distance = Dot(axes = 1, normalize=True)([encoded_left, encoded_right])
cos_similarity = Reshape((1,))(cos_distance)

model = Model(inputs=[left_input, right_input], outputs=[cos_similarity])

es_cb = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

model.compile(optimizer=Adam(lr=0.001),
              loss='mean_squared_error',
              metrics=['mae'])

history = model.fit([X_train['left'], X_train['right']], Y_train, epochs=n_epoch, batch_size=batch_size,
          validation_data=([X_validation['left'], X_validation['right']], Y_validation), callbacks=[es_cb])
#model.fit([X_train['left'], X_train['right']], train_labels, epochs=n_epoch, batch_size=batch_size, callbacks=[es_cb])



t0 = []
t1 = []
test = clean_data('./drive/MyDrive/Colab Notebooks/test_data/STS.input.images.txt')
test_df = pd.DataFrame({'left' : [], 'right' : []})
test_df['left'] = test[0]
test_df['right'] = test[1]

# Load training set
#for t in ['left', 'right']:
#    test_df[t + '_n'] = test_df[t]

X_test = test_df[['left', 'right']]
#X_test = split_and_zero_padding(X, max_seq_length)

test_labels = []
test_labels = gs_label("./drive/MyDrive/Colab Notebooks/test_data/STS.gs.images.txt")

Y_test = []
y_ts = []
for list in test_labels:
    for l in list:
        y_ts.append(float(l)/5)
Y_test = np.array(y_ts)

output_list = model.predict(x=[X_test['left'], X_test['right']], batch_size=len(X_test['left']))
print(output_list)
with open("./drive/MyDrive/Colab Notebooks/test_data/STS.output_RNN.images.txt", "w", encoding="utf-8") as fw:
    for w in output_list:
        w = str(w)
        w = w.replace("[", '')
        w = w.replace("]", '')
        print(w, file=fw)

KeyboardInterrupt: ignored